In [1]:
# data preprocessing modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy.stats import zscore

# ML modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, Input

# general stuff
import pandas as pd
import io
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# importing necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
# deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten , MaxPool2D
from tensorflow.keras import regularizers
# data preprocessing
from keras.utils import to_categorical


In [3]:
import tensorflow as tf
#import tensorflow_io as tfio
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
watermarked_DIR = '/Users/omaralmutoteh/Desktop/GitHub/audioseal/fixed_wavs_watermarkeds'
original_DIR = '/Users/omaralmutoteh/Desktop/GitHub/audioseal/fixed_wavs_originals'

In [5]:
def preprocess(file_path, label):

    file_contents = tf.io.read_file(file_path)
    audio, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    audio = tf.squeeze(audio, axis=-1)

    audio = audio[:16000] 


    spectrogram = tf.signal.stft(audio, frame_length=255, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, -1)
    spectrogram.set_shape([124, 129, 1])

    return spectrogram, label

In [6]:
import tensorflow as tf
import tensorflow_io as tfio
import os
from sklearn.model_selection import train_test_split


BATCH_SIZE = 32
SEED = 123
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.1

# Step 1: Load file paths
file_paths_original = tf.io.gfile.glob(os.path.join(original_DIR, '*.wav'))
file_paths_watermarked = tf.io.gfile.glob(os.path.join(watermarked_DIR, '*.wav'))
labels = [0] * len(file_paths_original) + [1]*len(file_paths_watermarked) # All original files = label 0
file_paths = file_paths_original+file_paths_watermarked

# Step 2: Split into train, val, test
train_files, temp_files, train_labels, temp_labels = train_test_split(
    file_paths, labels, test_size=VALIDATION_SPLIT + TEST_SPLIT, stratify=labels, random_state=SEED
)
val_size = int(VALIDATION_SPLIT / (VALIDATION_SPLIT + TEST_SPLIT) * len(temp_files))
val_files, test_files = temp_files[:val_size], temp_files[val_size:]
val_labels, test_labels = temp_labels[:val_size], temp_labels[val_size:]

In [7]:
def create_dataset(file_paths, labels):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
    return ds

In [8]:
train_ds = create_dataset(train_files, train_labels)
val_ds = create_dataset(val_files, val_labels)
test_ds = create_dataset(test_files, test_labels)

In [9]:
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras import regularizers, losses, optimizers

model1 = Sequential()

model1.add(Conv2D(filters=4, padding='same', activation='relu', kernel_size=(5,5), input_shape=(124,129,1)))
model1.add(MaxPool2D(pool_size=(2,2), strides=2))
model1.add(Conv2D(filters=8, padding='same', activation='relu', kernel_size=(5,5)))
model1.add(MaxPool2D(pool_size=(2,2), strides=2))
model1.add(Conv2D(filters=16, padding='same', activation='relu', kernel_size=(5,5)))
model1.add(MaxPool2D(pool_size=(2,2), strides=2))
model1.add(Flatten())
model1.add(Dense(50, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(1, activation="sigmoid"))

model1.summary()


model1.compile(
    loss=losses.BinaryCrossentropy(),  
    optimizer=optimizers.SGD(),
    metrics=["accuracy"]  
)


/Users/omaralmutoteh/anaconda3/envs/AudioSeal/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 124, 129, 4)    │           104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 64, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 62, 64, 8)      │           808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 31, 32, 16)     │         3,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3840)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │       192,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 196,229 (766.52 KB)

 Trainable params: 196,229 (766.52 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.5082 - loss: 1.6603 - val_accuracy: 0.4877 - val_loss: 1.5970
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.5119 - loss: 1.5737 - val_accuracy: 0.4843 - val_loss: 1.5179
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.5171 - loss: 1.4953 - val_accuracy: 0.4692 - val_loss: 1.4456
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.5253 - loss: 1.4241 - val_accuracy: 0.4644 - val_loss: 1.3800
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.5269 - loss: 1.3592 - val_accuracy: 0.4592 - val_loss: 1.3202
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.5246 - loss: 1.3002 - val_accuracy: 0.4573 - val_loss: 1.2659
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.5274 - loss: 1.2464 - val_accuracy: 0.4549 - val_loss: 1.2162
Epoch 8/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.5312 - loss: 1.1973 - v

In [12]:
# evaluate test accuracy
score = model1.evaluate(test_ds, verbose=0)
accuracy = 100*score[1]
# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

Test accuracy 46.49%
